In [126]:
from pspnet import PSPNet
from tensorflow.keras.models import load_model
import torchvision.transforms as std_trnsf
import cv2
import numpy as np
import torch
import tensorflow as tf

In [130]:
#segmentation+model 환경 가져오기
net = PSPNet(num_class=1, base_network='resnet101').to('cpu')
state = torch.load('D:/data-anal/marytalk/model/pspnet_resnet101_sgd_lr_0.002_epoch_100_test_iou_0.918.pth',map_location={'cuda:0': 'cpu'})
image_transforms = std_trnsf.Compose([
std_trnsf.ToTensor(),
std_trnsf.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
model = load_model('model/segmentation1.hd5')

In [129]:
def imageClassifier(imgpath :str):
    img = cv2.imread(imgpath)
    if img is None :
        #print('사진이 이미지 아님')
        return
    
    data = image_transforms(img)
    data = torch.unsqueeze(data, dim=0)
    net.load_state_dict(state['weight'])
    net.eval()
    logit = net(data)
    pred = torch.sigmoid(logit.cpu())[0][0].data.numpy()
    mask = pred >=0.5
    img[mask!=True]=255
    
    #img classfication
    categories = ['baby_perm', 'bob_cut', 'buzz_cut', 'c_curl_perm', 'dandy_cut', 'dreadlocks', 'hippie_perm', 'hush_cut', 'layered_cut', 'mohican', 'part_perm', 'pomade', 'wave_perm', 'wolf_cut']
    img = cv2.resize(img, dsize=(224,224))
    data= np.expand_dims(img,axis=0)
    data =data.astype('float32')
    with tf.device('/CPU:0'):
        preds = model.predict(data)
        if len(preds>=0.7)==0 :
            return ''
        else :
            a = categories[np.argmax(preds)]
            b = categories[np.argsort(preds[0])[-2]]
            c = categories[np.argsort(preds[0])[-3]]
            return a, b, c

pomade dandy_cut dreadlocks
